In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib



In [3]:
class HyperParametersSummary:
    def __init__(self, filter_path: str, test_fname='final_test_miou.txt'):
        self.predictions = []
        self.parameters = []
        self.table = []
        self.test_fname = test_fname
        for f in tqdm(glob(filter_path)):
            parameters = f.split('/')[-1]
            self.parameters.append(parameters)
            column = {}
            for parameter in parameters.split(','):                
                parameter_key, parameter_value = parameter.split('=')
                column[parameter_key] = parameter_value
            # Insert some code here to parse the metrics and their values
            test_file = os.path.join(f, 'testing', "fo=%s" %column["fold"], self.test_fname)
            if os.path.isfile(test_file):
                with open(test_file) as f:
                    content = f.readlines()
                    if len(content) > 0:
                        content = content[0].split(',')
                        for line in content[:1]:
                            key, value = line.split(':')
                            column[key] = float(value)
            else:
                pass
            self.table.append(column)
        self.table = pd.DataFrame(self.table)
        self.group_names = ', '.join(self.table.columns)
        print(f'Parameters: {self.group_names}')


## MS COCO

In [4]:
exp_dir = "experiments/200113_152702_n_shots_fold_embed_type_bs_seed_milestone_length_num_epoch_split_model_type_dataset_name_coco_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')
results_aggregated1 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated1['mIOU'] = results_aggregated1.mean(axis=1)

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_5.txt')
results_aggregated5 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated5['mIOU'] = results_aggregated5.mean(axis=1)
results_aggregated1.fillna(0.0)+results_aggregated5.fillna(0.0)

100%|██████████| 160/160 [00:00<00:00, 3965.82it/s]

Parameters: milestone_length, num_epoch, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU
Parameters: milestone_length, num_epoch, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU                             \
fold                                       0        1        2        3   
dataset_name n_shots model_type                                           
coco         1       coatt           0.18464  0.20430  0.12562  0.12638   
                     iter_nwe_coatt  0.19110  0.20296  0.10932  0.09836   
                     nwe             0.17058  0.19338  0.14904  0.10012   
                     nwe_coatt       0.14858  0.18116  0.13042  0.11114   
             5       coatt           0.18338  0.20570  0.12452  0.12890   
                     iter_nwe_coatt  0.20484  0.20254  0.11080  0.10408   
                     nwe             0.17810  0.19100  0.14144  0.10656   
                     nwe_coatt       0.14770  0.17332  0.12954  0.10932   

                                         mIOU  
fold                                           
dataset_name n_shots model_type                
coco         1       coatt           0.160235  
                     iter_nwe_coatt  0.150435  
                     nwe             0.153280  
                     nwe_coatt       0.142825  
             5       coatt           0.160625  
                     iter_nwe_coatt  0.155565  
                     nwe             0.154275  
                     nwe_coatt       0.139970

## PASCAL

In [5]:
exp_dir = "experiments/200113_145517_n_shots_fold_embed_type_bs_seed_milestone_length_split_model_type_dataset_name_pascal_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')

results_aggregated1 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated1['mIOU'] = results_aggregated1.mean(axis=1)

100%|██████████| 160/160 [00:00<00:00, 632.33it/s]

Parameters: milestone_length, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


In [6]:
exp_dir = "experiments/200113_145517_n_shots_fold_embed_type_bs_seed_milestone_length_split_model_type_dataset_name_pascal_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_5.txt')

results_aggregated5 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated5['mIOU'] = results_aggregated5.mean(axis=1)
results_aggregated1.fillna(0.0)+results_aggregated5.fillna(0.0)

100%|██████████| 160/160 [00:00<00:00, 733.14it/s] 

Parameters: milestone_length, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU                             \
fold                                       0        1        2        3   
dataset_name n_shots model_type                                           
pascal       1       coatt           0.40418  0.57178  0.43566  0.36582   
                     iter_nwe_coatt  0.42510  0.64758  0.48116  0.46464   
                     nwe             0.44928  0.65242  0.47472  0.47286   
                     nwe_coatt       0.42622  0.65386  0.47590  0.45756   
             5       coatt           0.43398  0.63422  0.46002  0.43722   
                     iter_nwe_coatt  0.45890  0.65722  0.48610  0.46556   
                     nwe             0.45538  0.65436  0.47268  0.47510   
                     nwe_coatt       0.45698  0.65632  0.47850  0.44060   

                                         mIOU  
fold                                           
dataset_name n_shots model_type                
pascal       1       coatt           0.444360  
                     iter_nwe_coatt  0.504620  
                     nwe             0.512320  
                     nwe_coatt       0.503385  
             5       coatt           0.491360  
                     iter_nwe_coatt  0.516945  
                     nwe             0.514380  
                     nwe_coatt       0.508100